In [27]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Reshape, merge, RepeatVector

batch_size = 10
nb_epoch = 50

# input features
nb_features = len(features)

# convert class vectors to binary class matrices
#Y_train = np_utils.to_categorical(y_train, nb_classes)

input_vec = Input(shape=(nb_features,), dtype='float32', name='input_vec')

x = Dense(16, activation='tanh')(input_vec)
x = Dense(32, activation='tanh')(x)
x = Dropout(0.25)(x)
x = Dense(32, activation='tanh')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(input=input_vec, output=x)

model.compile(loss='binary_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
                 verbose=1, validation_data=None)
#score = model.evaluate(X_test, Y_test, verbose=0)
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

Epoch 1/50
891/891 [==============================] - 0s - loss: 0.6524 - acc: 0.6442     
Epoch 2/50
891/891 [==============================] - 0s - loss: 0.5684 - acc: 0.7239     
Epoch 3/50
891/891 [==============================] - 0s - loss: 0.5311 - acc: 0.7576     
Epoch 4/50
891/891 [==============================] - 0s - loss: 0.5088 - acc: 0.7654     
Epoch 5/50
891/891 [==============================] - 0s - loss: 0.5113 - acc: 0.7565     
Epoch 6/50
891/891 [==============================] - 0s - loss: 0.4838 - acc: 0.7778     
Epoch 7/50
891/891 [==============================] - 0s - loss: 0.4818 - acc: 0.7834     
Epoch 8/50
891/891 [==============================] - 0s - loss: 0.4927 - acc: 0.7778     
Epoch 9/50
891/891 [==============================] - 0s - loss: 0.4667 - acc: 0.7946     
Epoch 10/50
891/891 [==============================] - 0s - loss: 0.4742 - acc: 0.7879     
Epoch 11/50
891/891 [==============================] - 0s - loss: 0.4614 - acc: 0.7834   

In [22]:
from data import Map, Select, identity
import math

features = ['Sex', 'Age', 'Age_invalid', 'Pclass', 'Fare_invalid', 'Fare']

feature_infos = [
    {
        'names' : ["Sex"],
        'func' : lambda x : (0,) if x == "male" else (1,),
    },
    {
        'from_names' : ["Age"],
        'to_names' : ["Age_invalid", "Age"],
        'func' : lambda x : (0, x) if 0 <= x <= 100 else (1, 0),
    },
    {
        
        'from_names' : ["Fare"],
        'to_names' : ["Fare_invalid", "Fare"],
        'func' : lambda x : (0, math.log(x)) if 0 < x else (1, 0),
    },
]        

def parse(data, features, feature_infos=[]) :
    to_data = dict(data)
    for info in feature_infos :
        to_data.update(Map(**info)(data))
    
    l_data = list()
    for feat in features :
        l_data.append(to_data[feat])
    npdata = np.array(l_data)
    return npdata.astype('float32').transpose()

def get_data(filename, features, feature_infos=[]) :
    data = pd.read_csv(filename)
    npdata = parse(data, features, feature_infos)
    return npdata
    
path = "../../data/original/"
train_filename = path + "train.csv"
test_filename = path + "test.csv"

X_train = get_data(train_filename, features, feature_infos)
Y_train = get_data(train_filename, ['Survived'])
X_test  = get_data(test_filename,  features, feature_infos)
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (891, 6)
Y_train shape: (891, 1)
891 train samples
418 test samples
